In [1]:

import pandas as pd



In [2]:
df_m_0 = "min_0"
df_m_1 = "min_1"
df_m_2 = 'min_2'
# df_m_3 = pd.read_csv('../dataset/min_3.csv')
df_0 = 'informative_cluster_0'
df_1 = 'informative_cluster_1'
df_merged = 'merged_shuffled'
df_merged_labeled = 'merged_shuffled_labeled'
# df_2 = pd.read_csv('../dataset/informative_cluster_2.csv')
selected_file = df_m_0
df= pd.read_csv("../dataset/" + selected_file + ".csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3955 entries, 0 to 3954
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   type            3955 non-null   int64  
 1   amount          3955 non-null   float64
 2   oldbalanceOrg   3955 non-null   float64
 3   newbalanceOrig  3955 non-null   float64
 4   oldbalanceDest  3955 non-null   float64
 5   newbalanceDest  3955 non-null   float64
 6   isFraud         3955 non-null   int64  
dtypes: float64(5), int64(2)
memory usage: 216.4 KB


In [4]:
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,181.00,181.00,0.0,21182.00,0.00,1
1,1,2806.00,2806.00,0.0,26202.00,0.00,1
2,1,20128.00,20128.00,0.0,6268.00,12145.85,1
3,1,25071.46,25071.46,0.0,9083.76,34155.22,1
4,1,235238.66,235238.66,0.0,0.00,235238.66,1


In [5]:
from sdv.cag import SingleTableProgrammableConstraint

class TransactionBalanceLogic(SingleTableProgrammableConstraint):
    def is_valid(self, row):
        print(row)
        amount = row['amount']
        obo = row['oldbalanceOrg']
        nbo = row['newbalanceOrig']
        obd = row['oldbalanceDest']
        nbd = row['oldbalanceDest']

        # if any(v is None for v in [t, obo, nbo, obd]):
        #     return True  # skip NaNs
        return (obd >= 0) & (obo >= 0) & (nbo >= 0) & (amount>0) & (nbd >= 0)

    def transform(self, table_data):
        return table_data  # no transformation needed

    def get_updated_metadata(self, metadata):
        new_metadata = metadata.copy()
        return new_metadata

    def reverse_transform(self, table_data):
        return table_data


In [6]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import Metadata
metadata = Metadata.detect_from_dataframe(df)
synthesizer = CTGANSynthesizer(metadata)
synthesizer.add_constraints([TransactionBalanceLogic()])
synthesizer.fit(df)

I:\ETVOS\2nd semester\DS Lab 1\DSLAB-Project\venv1\Lib\site-packages\sdv\single_table\base.py:128: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


KeyboardInterrupt: 

In [10]:
data = synthesizer.sample(df.shape[0],output_file_path="../dataset/fake_" + selected_file + ".csv")

AssertionError: I:\ETVOS\2nd semester\DS Lab 1\DSLAB-Project\dataset\fake_min_0.csv already exists.